In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.fleet.utils.fleet_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
from shell.fleet.data.data_utilize import *
import logging
from sklearn.metrics import f1_score
import os
from shell.fleet.data.recv_utils import *
from tqdm import tqdm
import argparse
from functools import partial
from torchvision.utils import make_grid
from shell.utils.oodloss import OODSeparationLoss
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.zscore import ZSCORE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict
from experiments.run import handle_combine_modes

logging.basicConfig(level=logging.INFO)

In [23]:
experiment_folder = "."
experiment_name = "combine_modes_results/recv_data_no_sparse_False_recv_mod_add_data_backward_True_make_new_opt_True"
experiment_name = "combine_modes_results/gt_recv_data_no_sparse_False_recv_mod_add_data_backward_True_make_new_opt_True"
dataset = "kmnist"
algo = "modular"

use_contrastive = False
num_trains_per_class = 64 if dataset != "cifar100" else 256
seed = 0
parallel = False
task_id = 3


agent_id = 0

In [24]:
def modify_cfg(net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg):
    cfg = handle_combine_modes(cfg)
    return  net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg

In [25]:
save_dir = get_save_dir(experiment_folder, experiment_name,
                        dataset, algo, num_trains_per_class, use_contrastive, seed)

fleet = setup_fleet(save_dir, task_id=task_id, parallel=False, modify_cfg=modify_cfg)
# fleet = setup_fleet(save_dir, task_id=task_id, parallel=False, modify_cfg=modify_cfg)
fleet.change_save_dir('debug_combine_modes_results')

INFO:root:Setting seed 0


{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 10}, 'dataset': {'dataset_name': 'kmnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': False}, 'sharing_strategy': {'name': 'recv_data', 'num_coms_per_round': 2, 'num_queries': 20, 'num_data_neighbors': 5, 'num_filter_neighbors': 5, 'scorer': 'cross_entropy', 'query_score_threshold': 0.0, 'shared_memory_size': 32, 'comm_freq': 16, 'query_task_mode': 'current', 'pre_or_post_comm': 'post', 'sync_base': False, 'prefilter_strategy': 'oracle', 'add_data_prefilter_strategy': 'both', 'assign_labels_strategy': 'same_as_query', 'min_task': 0}, 'seed': 0, 'algo': 'modular', 'job_name': 'kmnist_modular_numtrain_64', 'num_agents': 8, 'root_save_dir': 'combine_modes_results/gt_recv

INFO:root:Class sequence: [5 0 4 3 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:Class sequence: [4 7 5 7 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (2000, 1, 28, 28)
INFO:root:task 2 :(128, 

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': False, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 4, 'use_contrastive': False}
<class 'shell.learners.er_dynamic.CompositionalDynamicER'>


In [26]:
agent = fleet.agents[agent_id]
if 'gt' not in experiment_name:
    comm = agent.communicator['recv_data']
    print(comm.neighbors)

In [27]:
if 'gt' not in experiment_name:
    fleet.communicate(task_id, end_epoch=None, comm_freq=None, num_epochs=None, strategy='recv_data')
else:
    fleet.communicate(task_id, end_epoch=None, comm_freq=None, num_epochs=None)

In [28]:
if 'gt' not in experiment_name:
    print(comm.incoming_data)
else:
    print(agent.incoming_data)

{}


In [29]:
agent.eval_test(task_id=task_id, include_avg=True)

include_avg True


{0: 0.8555, 1: 0.88, 2: 0.8575, 3: 0.89, 'avg': 0.87075}

In [33]:
agent.incoming_data

{}